In [ ]:
%matplotlib inline
from astropy.time import Time
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from lsst_efd_client import EfdClient
from collections import defaultdict
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from scipy.stats import norm
import os
# %load_ext lab_black # enable the Black extension for auto formatting
# %load_ext autoreload # enable auto-reload. Useful if we want to have a python module to import from
# %autoreload 2 # powering up autoreload

# Compare TMA balancing data from previous events with the next events

We are getting ready to balance the telescope twice in the next weeks. First, we will balance the telescope with ComCam and M2 Glass. The M2 Glass and M2 Surrogate have similar weights, with a small difference. We expect the torques applied by the elevation drives will be very close to the previous balancing event(s). A couple of weeks later, we will repeat the procedure with ComCam, M2 Glass, and M1M3 Glass. The M1M3 Glass and M1M3 Cell assembly is much heavier than the M1M3 Mass Simulator (yellow cross) and hundreds of kilograms heavier than the M1M3 Surrogate and M1M3 Cell configuration. This procedure will be much more delicate due to the size and mass of the mirror. 

We want to establish a baseline before we start the procedure, and we need someone to review the data to determine whether we can proceed quickly. 

The links below point to old night logs that might contain useful information. Feel free to unlink them if they are not useful. 

Here is an approximate timeline of different integration phases where we needed to re-balance the telescope. 
We do not necessarily need the whole process. We need the torques once the telescope is already balanced as a baseline.

May to Aug 2023 - M1M3 Surrogate and M1M3 Cell on the TMA

Nov 2023 to Jan 2024 - M1M3 Surrogate and Cell, M2 Surrogate and Cell on the TMA

Feb to Apr 2024 - M2 Surrogate and Cell on the TMA

## A few top-level imports (that I really should put in a library at some point...)

In [ ]:
async def query_bump_logs_in_chunks(
    start_date, end_date, client_name="", chunk_size_days=3,topic_name="lsst.sal.MTM1M3.logevent_logMessage",fields=["message"]
):
    """
    Queries the log messages related to bump tests from the EFD in chunks.

    Args:
        start_date (str): Start date of the query in ISO format (YYYY-MM-DD).
        
        end_date (str): End date of the query in ISO format (YYYY-MM-DD).
        
        client_name (str, optional): Name of the EFD client. Defaults to "".
        
        chunk_size_days (int, optional): Number of days per chunk. Defaults to 3.

        topic_name (str, optional): SAL topic name to be queried by the client. Defaults to lsst.sal.MTM1M3.logevent_logMessage.

        fields (list[str], optional): Fields to be queried by the client. Defaults to ["message"].

    Returns:
        pandas.DataFrame: Concatenated DataFrame containing the queried log messages.
    """

    client = makeClient(client_name)

    # Convert start and end dates to datetime objects
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)

    # Initialize an empty DataFrame to store concatenated results
    all_data = pd.DataFrame()

    current_start = start
    while current_start < end:
        current_end = min(current_start + timedelta(days=chunk_size_days), end)
        try:
            # Query the data for the current chunk
            chunk_data = await client.select_time_series(
                topic_name=topic_name,
                fields=fields,
                start=Time(current_start.isoformat(), format="isot", scale="utc"),
                end=Time(current_end.isoformat(), format="isot", scale="utc"),
            )
            # Concatenate the chunk data to the main DataFrame
            all_data = pd.concat([all_data, chunk_data], ignore_index=False)
        except Exception as e:
            print(
                f"Error querying data from {current_start.isoformat()} to {current_end.isoformat()}: {e}"
            )
            continue  # Optionally, continue to the next chunk

        # Move to the next chunk
        current_start = current_end

    return all_data

def makeClient(client_name):
        # Create the client based on client_name
    if client_name == "summit_efd":
        return makeEfdClient("summit_efd")
    elif client_name == "usdf_efd":
        return makeEfdClient("usdf_efd")
    elif client_name == "idf_efd":
        return makeEfdClient("idf_efd")
    else:
        return makeEfdClient()  # Default client


# Example usage:
# begin = "2023-11-13T01:00"
# end = "2023-12-21T01:00"
# bump_logs = await query_bump_logs_in_chunks(begin, end, client_name='')

def showAndClear():
    plt.show()
    # Clear the current axes.
    plt.cla() 
    # Clear the current figure.
    plt.clf() 
    # Closes all the figure windows.
    plt.close('all')   
    plt.close(fig)
    
    return

async def getDataFrame(client,starts,ends,topic,verbose=True,fields=None):
    
    all_data = pd.DataFrame()
    for start,end in zip(starts,ends):
        if verbose:
            print(r"Starting query for time range {} - {}".format(start,end),end=" . . . ")
        if fields != None:
            df_bump = await client.select_time_series(topic,fields, Time(start), Time(end))
        else:
            df_bump = await client.select_time_series(topic,"*", Time(start), Time(end))
        
        all_data = pd.concat([all_data, df_bump], ignore_index=False)

        del df_bump

        if verbose:
            print("Finished")
    
    return all_data

def makeDateRange(startPoint,endPoint,step=np.timedelta64(1, 'D')):
    starts = np.arange(startPoint,endPoint,step=step)
    ends = starts + np.timedelta64(1, 'D')
    return starts,ends

def fitGaussian(data,ax):
    mu, std = norm.fit(data) 
    
    xmin, xmax = ax.get_xlim()
    x = np.linspace(np.floor(xmin), np.ceil(xmax), int(10E4))
    p = norm.pdf(x, mu, std)

    return mu,std,p,x,xmin,xmax

def getFWHM_from_gaussian(sigma):
    return 2*np.sqrt(np.log(2)*2)*sigma

### Setting up the sub-directories

In [ ]:
base_dir = os.getcwd()
figure_dir,data_dir = base_dir+"/SITCOM-1508-plots",base_dir+"/SITCOM-1508-data"
for pathname in [figure_dir,data_dir]:
    if not os.path.isdir(pathname):
        os.mkdir(pathname)

In [ ]:
verbatim = False

## Get all topics, so we can take a look at what is here

In [ ]:
client = makeClient("usdf_efd")

a = await client.get_topics()

if verbatim:
    for entry in a:
        if entry.__contains__("lsst.sal.MTMount"):
            print(entry)

In [ ]:
b = await client.get_fields("lsst.sal.MTMount.elevation")

if verbatim:
    for entry in b:
        print(entry)

### M1M3 Surrogate and M1M3 Cell on the TMA - May 2023-Aug 2023

In [ ]:
startPoint = np.datetime64("2023-05-20T12:00:00")
endPoint = np.datetime64("2023-06-30T12:00:00")
starts,ends = makeDateRange(startPoint,endPoint)

topic_az = "lsst.sal.MTMount.azimuth"
fields_az = ['actualTorque']

topic_el = "lsst.sal.MTMount.elevation"
fields_el = ["actualPosition"]

In [ ]:
all_data_az = await getDataFrame(client,starts,ends,topic_az,fields=fields_az,verbose=False)
all_data_el = await getDataFrame(client,starts,ends,topic_el,fields=fields_el,verbose=False)

In [ ]:
booArray = np.diff(all_data_el["actualPosition"],prepend=1) == 0
timArray = all_data_el.index

In [ ]:
def getMaskedTimeBounds(boolArray,timeArray):
    '''
    A function to find the time periods of constant acceleration
    '''
    
    startEntry=True
    starts,ends = [],[]
    previous_time_entry = timeArray[0]
    
    for boolEntry,timeEntry in zip(boolArray,timeArray):
        if boolEntry and startEntry and not previous_bool_entry:
            starts.append(timeEntry)
            startEntry=False
            # print("appending to start")
        elif not boolEntry and not startEntry and previous_bool_entry:
            ends.append(previous_time_entry)
            startEntry=True
            # print("appending to end")
        previous_time_entry = timeEntry
        previous_bool_entry = boolEntry
    return starts, ends
            
constant_starts,constant_ends = getMaskedTimeBounds(booArray,timArray)

In [ ]:
lolim,uplim = pd.Timestamp("2023-05-29").tz_localize("UTC"),pd.Timestamp("2023-06-03").tz_localize("UTC")
limited_data = all_data_az.loc[lolim:uplim]
lolim,uplim = lolim.tz_localize(None),uplim.tz_localize(None)

In [ ]:
fig,axs = plt.subplots(1,2,figsize=[10,6],sharex=True)
axs[0].plot(limited_data.index,limited_data)
axs[1].hist(limited_data, bins=90, facecolor = '#2ab0ff', edgecolor='#169acf', linewidth=0.5)
mu,std,p,x,xmin,xmax = fitGaussian(limited_data,axs[1])
axs[1].plot(x, p,label="$\mu$={:.2E}, $\sigma$={:.2E}".format(mu, std), linewidth=2,color="red")
axs[0].set_ylabel("Torque [N m]")
axs[1].set_ylabel("Counts")
axs[0].set_xticks(np.arange(lolim,uplim,step=np.timedelta64(1,"D")))
axs[0].set_xlim(lolim,uplim)
for ax in axs:
    ax.grid()
# axs[0].set_ylim(-0.4E5,0.4E5)
# axs[1].set_ylim(-0.8E4,0.8E4)
axs[1].set_xlabel("Date")
showAndClear()

### M1M3 Surrogate and Cell, M2 Surrogate and Cell on the TMA - Nov 2023 to Jan 2024

In [ ]:
startPoint = np.datetime64("2023-11-01T12:00:00")
endPoint = np.datetime64("2024-01-01T12:00:00")
starts,ends = makeDateRange(startPoint,endPoint)

In [ ]:
all_data = await getDataFrame(client,starts,ends,topic,fields=fields,verbose=False)

In [ ]:
lolim,uplim = pd.Timestamp("2023-11-14").tz_localize("UTC"),pd.Timestamp("2023-11-25").tz_localize("UTC")
limited_data = all_data.loc[lolim:uplim]

In [ ]:
fig,axs = plt.subplots(2,1,figsize=[10,6],sharex=True)
for ax in axs:
    ax.plot(limited_data.index,limited_data)
    ax.set_ylabel("Torque [N m]")
    ax.set_xticks(np.arange(lolim,uplim,step=np.timedelta64(1,"D")))
    ax.set_xlim(lolim,uplim)
    ax.grid()
axs[0].set_ylim(-0.4E5,0.4E5)
axs[1].set_ylim(-0.8E4,0.8E4)
axs[1].set_xlabel("Date")
showAndClear()

### M2 Surrogate and Cell on the TMA - Feb to Apr 2024

In [ ]:
startPoint = np.datetime64("2024-02-01T12:00:00")
endPoint = np.datetime64("2024-04-01T12:00:00")
starts,ends = makeDateRange(startPoint,endPoint)

In [ ]:
all_data = await getDataFrame(client,starts,ends,topic,fields=fields,verbose=False)

In [ ]:
lolim,uplim = pd.Timestamp("2024-02-05").tz_localize("UTC"),pd.Timestamp("2024-02-15").tz_localize("UTC")
limited_data = all_data.loc[lolim:uplim]

In [ ]:
fig,axs = plt.subplots(2,1,figsize=[10,6],sharex=True)
for ax in axs:
    ax.plot(limited_data.index,limited_data)
    ax.set_ylabel("Torque [N m]")
    ax.set_xticks(np.arange(lolim,uplim,step=np.timedelta64(2,"D")))
    ax.set_xlim(lolim,uplim)
    ax.grid()
axs[0].set_ylim(-2E5,2E5)
axs[1].set_ylim(-0.8E2,0.8E2)
axs[1].set_xlabel("Date")
showAndClear()